In [ ]:
import os
from datetime import date
from datetime import datetime

import pandas as pd

In [ ]:
from finlab import data

In [ ]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings
from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')  # Google SpreadSheet 表單ID
OUTPUT_GSHEET_NAME = '選股清單03'

In [ ]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')

## 外部資料讀取

In [ ]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
institutional_investors_foreign = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)', save_to_storage=True)
institutional_investors_inv_trust = data.get('institutional_investors_trading_summary:投信買賣超股數', save_to_storage=True)
institutional_investors_dealer = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)', save_to_storage=True)

# broker_info = data.get('broker_mapping', save_to_storage=True)
top15_broker = data.get('broker_transactions', save_to_storage=True)
inventory = data.get('inventory', save_to_storage=True)

## 數據分析

In [ ]:
vol.fillna(0.0, inplace=True)
vol_one_lot = vol / 1000
vol_one_lot = vol_one_lot.round().astype(int)  # 成交股數轉成張數
vol_one_lot.tail(1)

In [ ]:
# close["3228"]["2023-11-21":"2023-11-21"]
# vol["3228"]["2023-11-21":"2023-11-21"]
# vol_one_lot["2330"]

In [ ]:
# inventory_level = {
#     1: "1-999",
#     2: "1,000-5,000",
#     3: "5,001-10,000",
#     4: "10,001-15,000",
#     5: "15,001-20,000",
#     6: "20,001-30,000",
#     7: "30,001-40,000",
#     8: "40,001-50,000",
#     9: "50,001-100,000",
#     10: "100,001-200,000",
#     11: "200,001-400,000",
#     12: "400,001-600,000",
#     13: "600,001-800,000",
#     14: "800,001-1,000,000",
#     15: "1,000,001以上",
#     17: "合計",
# }

In [ ]:
# close.tail(1)
inventory[inventory['date'] == inventory['date'].max()]

In [ ]:
institutional_investors_foreign.tail(1)

In [ ]:
# top15_broker
top15_broker["net_buy"] = top15_broker["buy"] - top15_broker["sell"]
# top15_broker[(top15_broker["stock_id"] == "00903") & (top15_broker["date"] == top15_broker["date"].max())].sort_values(by="net_buy", ascending=False)
# top15_broker[(top15_broker["stock_id"] == "2330") & (top15_broker["date"] == "2024-06-28")]

In [ ]:
# 針對top15_broker做pivot table彙總，以欄位stock_id, date進行aggregate sum(net_buy)
top15_broker_summary_daily = top15_broker.pivot_table(index=["stock_id"], columns=["date"], values="net_buy", aggfunc='sum').T
top15_broker_summary_daily.fillna(0.0, inplace=True)

In [ ]:
top15_broker_summary_daily.tail(1)

In [ ]:
# 主力籌碼集中度 （1日、5日、10日）
major_player_chips_concentration_cum1 = top15_broker_summary_daily / vol_one_lot * 100
major_player_chips_concentration_cum5 = top15_broker_summary_daily.rolling(window=5).sum() / vol_one_lot.rolling(window=5).sum() * 100
major_player_chips_concentration_cum10 = top15_broker_summary_daily.rolling(window=10).sum() / vol_one_lot.rolling(window=10).sum().tail(1) * 100

In [ ]:
# # 顯示個股主力籌碼集中度
# sid = "6829"
# print(major_player_chips_concentration_cum1[sid].tail(1))
# print(major_player_chips_concentration_cum5[sid].tail(1))
# print(major_player_chips_concentration_cum10[sid].tail(1))

In [ ]:
# 選股條件
cond1 = major_player_chips_concentration_cum1 > 20  # 主力籌碼集中度1日 > 10%
cond2 = major_player_chips_concentration_cum5 > 20  # 主力籌碼集中度5日 > 10%
cond3 = major_player_chips_concentration_cum10 > 20  # 主力籌碼集中度10日 > 10%

filter_stock = (cond1 & cond2 & cond3).tail(1)
# filter_stock = (cond2 & cond3).tail(1)

In [ ]:
filter_stock = filter_stock.tail(1)
filtered_symbols = filter_stock.columns[filter_stock.iloc[0]].tolist()
df_filtered_symbols = pd.DataFrame(filtered_symbols, columns=['symbol'])

print(df_filtered_symbols.shape)
df_filtered_symbols.head(3)

In [ ]:
stock_name = stock_info[['stock_id', '公司簡稱']]
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

In [ ]:
# stock_name[stock_name['symbol'] == '5483']

In [ ]:
# Assuming you have a DataFrame called stock_name with columns stock_id and stock_name

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = df_filtered_symbols.merge(stock_name, on='symbol', how='left')
merged_df = merged_df.fillna('', inplace=False)
merged_df["web_link"] = merged_df["symbol"].apply(lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart")
merged_df["題材概念股"] = merged_df["symbol"].apply(lambda x: tst.read_topic_stocks(x))

# Print the merged DataFrame
print(merged_df)

In [ ]:
# # major_player_chips_concentration_cum1[filtered_symbols].tail(1)
# major_player_chips_concentration_cum10[filtered_symbols].tail(1)

In [ ]:
# 輸出報表留存
today = datetime.now().strftime("%Y%m%d")
merged_df.to_excel(f'{REPORT_PATH}/{OUTPUT_GSHEET_NAME}_{today}.xlsx', index=False)

## 輸出結果至Google sheet

In [ ]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

In [ ]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [ ]:
# 更新工作表資料
gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY, 
    OUTPUT_GSHEET_NAME, 
    [merged_df.columns.values.tolist()] + merged_df.values.tolist()
)